In [1]:
import pandas as pd
import numpy as np
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [2]:
import warnings

# ignore all warnings
warnings.filterwarnings("ignore")

# code that may trigger a warning

In [3]:
df=pd.read_csv('data.csv', encoding= 'unicode_escape')

In [4]:
df['Description']=df['Description'].str.strip()
df.dropna(axis=0,subset=['InvoiceNo'],inplace=True)
df['InvoiceNo']=df['InvoiceNo'].astype('str')
df=df[~df['InvoiceNo'].str.contains('C')]

In [5]:
df['Country'].value_counts()

United Kingdom          487622
Germany                   9042
France                    8408
EIRE                      7894
Spain                     2485
Netherlands               2363
Belgium                   2031
Switzerland               1967
Portugal                  1501
Australia                 1185
Norway                    1072
Italy                      758
Channel Islands            748
Finland                    685
Cyprus                     614
Sweden                     451
Unspecified                446
Austria                    398
Denmark                    380
Poland                     330
Japan                      321
Israel                     295
Hong Kong                  284
Singapore                  222
Iceland                    182
USA                        179
Canada                     151
Greece                     145
Malta                      112
United Arab Emirates        68
European Community          60
RSA                         58
Lebanon 

In [15]:
invoices=(df[df['Country']=='Poland']
          .groupby(['InvoiceNo','Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [16]:
def my_encode_units(x):
    if x <=0:
        return 0
    else:
        return 1
invoices_sets=invoices.applymap(my_encode_units)

In [17]:
try:
    invoices_sets.drop('POSTAGE', inplace=True, axis=1)
except Exception as e:
    pass

In [18]:
model=apriori(invoices_sets,min_support=0.07,use_colnames=True)

In [ ]:
my_rules=association_rules(model,metric='lift',min_threshold=1)

In [ ]:
x=my_rules[(my_rules['lift']>=3)&
          (my_rules['confidence']>=0.5)]

In [ ]:
x['antecedents'] = x['antecedents'].map(lambda x: list(x))
x['antecedents'] = x['antecedents'].map(lambda x: x[0])
x['consequents'] = x['consequents'].map(lambda x: list(x))
x['consequents'] = x['consequents'].map(lambda x: x[0])
x=x.drop_duplicates(subset=['antecedents','consequents'],keep='last')

In [ ]:
def find_combos(item):
    res=x[x['antecedents']==item].reset_index()
    return res[['consequents','confidence','lift','support']]

In [ ]:
find_combos('PLASTERS IN TIN CIRCUS PARADE')


In [ ]:
x